# On essaie d'ouvrir les CSV fournis par Eric

Consigne : 

> trouver la variable "CYCLE" et définir la donnée correspondante à la date du 13/10/2014 13:09 par exemple

In [ ]:
import csv 

fname = "./jour3/RechercheCSV.csv"

def ouverture_avec_csv():
    with open(fname) as f_csv:
        csv_re